# MasterMonitor Sensor Script

## AdaFruit IO

### Credentials

In [1]:
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

load_dotenv()

AdaFruitIO_Username = os.environ.get("ADAFRUIT_IO_USERNAME")
AdaFruitIO_Key = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(AdaFruitIO_Username, AdaFruitIO_Key)

### Feeds

In [2]:
def getFeed(name):
    try:
        feedID = aio.feeds(name)
    except RequestError:
        feed = Feed(name=name)
        feedID = aio.create_feed(feed)
    return feedID

soundFeed = getFeed("sound")
luxFeed = getFeed("light")
intTempFfeed = getFeed("inttempf")
extemp2Feed = getFeed("extemp2")
humidFeed = getFeed("humidity")
pressFeed = getFeed("pressure")
tvocsFeed = getFeed("new-sensor.tvocs")
co2Feed = getFeed("new-sensor.co2")
msgFeed = getFeed("new-sensor.messages")

## Google Sheets

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "enviro-pi_log"

scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

sheet = client.open(sheet_name).worksheet("Values")

## BME280 Sensor

In [4]:
from bme280 import BME280

bme280 = BME280()

## External Temperature Sensor

In [5]:
import smbus

def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

## CCS811 Sensor

In [6]:
import busio
from board import I2C
from adafruit_ccs811 import CCS811

i2c = I2C() 
ccs = CCS811(i2c)

while (not ccs.data_ready):
    pass

print ("CCS811 Sensor ready.")

CCS811 Sensor ready.


## Light Sensor

In [7]:
from ltr559 import LTR559

ltr559 = LTR559()

## Noise Sensor

In [8]:
from enviroplus.noise import Noise

noise = Noise()

## LCD Screen

### Imports

In [9]:
import ST7735
from PIL import Image, ImageDraw, ImageFont
from fonts.ttf import RobotoMedium as UserFont

### Code

In [10]:
disp = ST7735.ST7735(port=0,cs=1,dc=9,backlight=12,rotation=270,spi_speed_hz=10000000)
disp.begin()

back_color = ( 46, 139,  87)
text_color = (255, 160, 122)
head_color = (250, 128, 114)
warn_color = (220,  20,  60)
font_size  = 14
text_margin = 5

img = Image.new('RGBA', (disp.width, disp.height))
draw = ImageDraw.Draw(img)
font = ImageFont.truetype(UserFont, font_size)
font_big = ImageFont.truetype(UserFont, 2*font_size)
size_x, size_y = draw.textsize('text', font)

def updateLCD(label, value, unit): 
    if (value < 1000):
        vstr="{0:.1f} {1}".format(value, unit)
    else:
        vstr="{0:.0f} {1}".format(value, unit)
        
    draw.rectangle((0, 0, disp.width, disp.height), back_color)
    draw.text((text_margin, text_margin), "Berry_Blob", font=font, fill=head_color)
    draw.text((text_margin, text_margin+2.0*size_y), label, font=font, fill=text_color)
    draw.text((text_margin, text_margin+3.5*size_y), vstr, font=font_big, fill=text_color)
    disp.display(img)

## Main

### Imports

In [11]:
import time
from IPython.display import clear_output
from board import *
from datetime import datetime

### Variables

In [12]:
metadata = {'lat': 36.017433,
       'lon': -84.239815,
       'ele': 0,
       'created_at': None}

update_data = 1200.0
update_msg  = 1800.0
update_lcd  = 3.0
next_lcd    = 'intc'
count = 0

running = True

tvoc_alert = False
tvoc_alert_level = 1500

dashboard_url = "https://io.adafruit.com/Thuviksa/dashboards/weather-monitor"

### Loop

In [ ]:
start_time   = time.time()
current_time = start_time

count = 0

while running:
    try:
        if ((int (current_time - start_time) % update_data) == 0):
            interc = bme280.get_temperature()
            interf = interc * 1.8 + 32
            humid = bme280.get_humidity()
            press = bme280.get_pressure()
            co2 = ccs.eco2
            tvocs = ccs.tvoc
            lux = ltr559.get_lux()
            prox = ltr559.get_proximity()
            range=[(100, 1200)]
            amps = noise.get_amplitudes_at_frequency_ranges(range)
            extc = readTemp()
            extf = extc * 1.8 + 32

            ltr559.update_sensor()
            output = "External-Temp: {0:.2f} C - {1:.2f} F".format(extc, extf)
            print(output)
            output = "Internal-Temp: {0:.2f} C - {1:.2f} F".format(interc, interf)
            print(output)
            output = "Humdity: {0:.2f} % - Pressure: {1:.2f} hPa".format(humid, press)
            print(output)
            output = "Light: {0:.2f} lux - Noise: {1:.2f} - Proximity: {2:.2f}".format(lux, amps[0], prox)
            print(output)
            output = "CO2: {0:.2f} ppm - TVOCs: {1:.2f} ppb".format(co2, tvocs)
            print(output)
            print('Updated: ', datetime.now())
            
            aio.send_data(co2Feed.key, co2, metadata)
            aio.send_data(tvocsFeed.key, tvocs, metadata)
            aio.send_data(soundFeed.key, amps[0], metadata)
            aio.send_data(luxFeed.key, lux, metadata)
            aio.send_data(intTempFfeed.key, interf, metadata)
            aio.send_data(extemp2Feed.key, extf, metadata)
            aio.send_data(humidFeed.key, humid, metadata)
            aio.send_data(pressFeed.key, press, metadata)
            print("Sending Data to the AdaFruit Cloud... 📊📊📊")
            
            now = datetime.now()
            nowtime = now.strftime("%H:%M:%S")
            nowday = now.strftime("%m/%d/%Y")
            row = [nowday, nowtime, interc, interf, extc, extf, humid, press, lux, amps[0], co2, tvocs]
            sheet.append_row(row)
            
            if(tvocs > tvoc_alert_level):
                if(not tvoc_alert):
                    tvoc_alert=True
                    msg="TVOC Reading is over *{0:.2f}* ppb ⚠⚠⚠".format(tvoc_alert_level)
                    aio.send_data(msgFeed.key, msg, metadata)
            else:
                if(tvoc_alert):
                    tvoc_alert=False
                    msg="TVOC Reading returned to below *{0:.2f}* ppb 😀😃😃 ".format(tvoc_alert_level)
                    aio.send_data(msgFeed.key, msg, metadata)
        
        count = count + 1
        
        if (count > update_data):
            print("Update message")
            msg = "_Thuviksa's Updates_"
            msg+="<br>Internal-Temp: *{0:.2f} C* - *{1:.2f} F*".format(interc,interf)
            msg+="<br>External-Temp: *{0:.2f} C* - *{01:.2f} F*".format(extc, extf)
            msg+="<br>Humidity: *{0:.2f} %* - Pressure: *{1:.2f} hPa*".format(humid,press)
            msg+="<br>Air quality: CO2 - *{0:.2f} ppm*, TVOC - *{1:.2f} ppb*<br>".format(co2,tvocs)
            msg+="<br>Light: *{0:.2f} lux* - Noise *{1:.2f}* - Prox *{2:.2f}*<br>".format(lux,amps,prox)
            msg+="<br> "
            msg+="<br>Full dashboard: {}".format(dashboard_url)
            aio.send_data(msgFeed.key, msg, metadata)
        
        time.sleep(update_lcd)
        current_time = time.time()
        
        if (next_lcd == 'intc'):
            updateLCD('Internal-Temp', interc, 'C')
            next_lcd = 'intf'
        elif (next_lcd == 'intf'):
            updateLCD('Internal-Temp', interf, 'F')
            next_lcd = 'extc'
        elif (next_lcd == 'extc'):
            updateLCD('External-Temp', extc, 'C')
            next_lcd = 'extf'
        elif (next_lcd == 'extf'):
            updateLCD('External-Temp', extf, 'F')
            next_lcd = 'humid'
        elif (next_lcd == 'humid'):
            updateLCD('Humidity', humid, '%')
            next_lcd = 'press'
        elif (next_lcd == 'press'):
            updateLCD('Pressure', press, 'hPa')
            next_lcd = 'lux'
        elif (next_lcd == 'lux'):
            updateLCD('Light', lux, 'lux')
            next_lcd = 'amps'
        elif (next_lcd == 'amps'):
            updateLCD('Noise', amps[0], 'Hz')
            next_lcd = 'co2'
        elif (next_lcd == 'co2'):
            updateLCD('CO2', co2, 'ppm')
            next_lcd = 'tvocs'
        elif (next_lcd == 'tvocs'):
            updateLCD('TVOCs', tvocs, 'ppb')
            next_lcd = 'prox'
        elif (next_lcd == 'prox'):
            updateLCD('Proximity', prox, ' ')
        
        clear_output(wait=True)
        
    except IOError:
        print("Just an IO Error, nothing to worry 😉😉😉")
        pass
    
    except KeyboardInterrupt:
        print("Loop Terminated by User 👾👾👾")
        running = False
        
    except RuntimeError:
        print("Not Good  👎👎👎")
        pass